In [ ]:
import numpy as np
import talib

class Portfolio:
    def __init__(self, initial_capital):
        self.capital = initial_capital
        self.position = 0
        self.trade_history = []
    
    def buy(self, price, amount):
        cost = price * amount
        if cost <= self.capital:
            self.capital -= cost
            self.position += amount
            self.trade_history.append(('BUY', price, amount))
        else:
            print("Not enough capital to buy.")
    
    def sell(self, price, amount):
        if amount <= self.position:
            self.capital += price * amount
            self.position -= amount
            self.trade_history.append(('SELL', price, amount))
        else:
            print("Not enough position to sell.")
    
    def get_value(self, market_price):
        return self.capital + self.position * market_price


In [ ]:
class Backtest:
    def __init__(self, data, initial_capital):
        self.data = data
        self.initial_capital = initial_capital
        self.portfolio = Portfolio(initial_capital)
    
    def run_strategy(self, buy_signal, sell_signal):
        for i in range(1, len(self.data)):
            if buy_signal[i]:
                self.portfolio.buy(self.data['price'][i], 1)
            elif sell_signal[i]:
                self.portfolio.sell(self.data['price'][i], 1)
    
    def calculate_pnl(self):
        final_value = self.portfolio.get_value(self.data['price'].iloc[-1])
        pnl = final_value - self.initial_capital
        return pnl
    
    def calculate_sharpe_ratio(self, risk_free_rate=0.01):
        daily_returns = self.data['price'].pct_change()
        excess_returns = daily_returns - risk_free_rate / 252
        sharpe_ratio = np.sqrt(252) * excess_returns.mean() / excess_returns.std()
        return sharpe_ratio


In [1]:
import pandas as pd

price_path = ".\\data\\gold\\Prices.csv"
goldPrice = pd.read_csv(price_path)

goldPrice = goldPrice[['Date', 'USD']]
goldPrice['Date'] = pd.to_datetime(goldPrice['Date'])
goldPrice.set_index('Date', inplace=True)
goldPrice = goldPrice[(goldPrice.index >= '1980-01-01') & (goldPrice.index <= '2023-12-31')]

In [ ]:
moving_average_short = talib.SMA(goldPrice['price'], timeperiod=40) # 短期移动平均
moving_average_long = talib.SMA(goldPrice['price'], timeperiod=100) # 长期移动平均

buy_signal = (moving_average_short.shift(1) < moving_average_long.shift(1)) & (moving_average_short > moving_average_long) # 买入信号：短期移动平均线上穿长期移动平均线
sell_signal = (moving_average_short.shift(1) > moving_average_long.shift(1)) & (moving_average_short < moving_average_long) # 卖出信号：短期移动平均线下穿长期移动平均线

initial_capital = 10000
backtest = Backtest(goldPrice, initial_capital)
backtest.run_strategy(buy_signal, sell_signal)

pnl = backtest.calculate_pnl()
sharpe_ratio = backtest.calculate_sharpe_ratio()

print(f"Strategy PnL: {pnl}")
print(f"Sharpe Ratio: {sharpe_ratio}")